In [4]:
import numpy as np
from patsy import dmatrix
import statsmodels.api as sm
from numpy.linalg import inv
# global cubic
from numpy.linalg import inv
from numpy.linalg import multi_dot
from src.covariances import COV as cov_fn


# generate data

In [2]:

np.random.seed(42)
x = np.random.uniform(0, 1, 50)
x = np.sort(x)
err = np.random.normal(0, 1, 50)
y = np.add(x, err)

## Calculate point variances

In [5]:
cov_gl = cov_fn.GLCov(x, 1)
pt_var_gl = cov_gl[0][0] + cov_gl[1][1] * x * x + 2 * cov_gl[0][1] * x


x_square = x * x
x_cubic = x * x * x
cov_global_cubic = cov_fn.GlobalCubicCov(x, 1)
x_m = np.array([np.ones(len(x)), x, x * x, x * x * x]).transpose()
res = multi_dot([x_m, cov_global_cubic, x_m.transpose()])
pt_var_cubic = res.diagonal()

In [6]:
pt_var_gl 

array([0.06424158, 0.06141653, 0.05902431, 0.05678453, 0.05547465,
       0.04965829, 0.04565327, 0.04296265, 0.04055625, 0.04055283,
       0.03854755, 0.03705661, 0.03685319, 0.03666754, 0.03482901,
       0.03334285, 0.02856467, 0.02585208, 0.02578302, 0.02490893,
       0.02488322, 0.02440502, 0.021548  , 0.02124612, 0.02004779,
       0.02000815, 0.02002517, 0.02059323, 0.02114113, 0.02134435,
       0.02151974, 0.02248407, 0.02524782, 0.02570472, 0.0258897 ,
       0.02638785, 0.02673293, 0.03147281, 0.03388803, 0.03680563,
       0.04001263, 0.0481452 , 0.05213007, 0.05653427, 0.06318962,
       0.07251276, 0.08186273, 0.08231341, 0.08605084, 0.08714267])

In [7]:
# Fit a cubic spline with two knots at 0.33 and 0.66
x_cubic = dmatrix('bs(x, knots=(0.33, 0.66))', {'x': x})
fit_cubic = sm.GLM(y, x_cubic).fit()

# Fit a natural spline with lower and upper bounds
x_natural = dmatrix('cr(x, df=6, lower_bound=0.1, upper_bound=0.9)', {'x': x})
fit_natural = sm.GLM(y, x_natural).fit()





In [11]:
x_natural

DesignMatrix with shape (50, 7)
  Columns:
    ['Intercept',
     'cr(x, df=6, lower_bound=0.1, upper_bound=0.9)[0]',
     'cr(x, df=6, lower_bound=0.1, upper_bound=0.9)[1]',
     'cr(x, df=6, lower_bound=0.1, upper_bound=0.9)[2]',
     'cr(x, df=6, lower_bound=0.1, upper_bound=0.9)[3]',
     'cr(x, df=6, lower_bound=0.1, upper_bound=0.9)[4]',
     'cr(x, df=6, lower_bound=0.1, upper_bound=0.9)[5]']
  Terms:
    'Intercept' (column 0)
    'cr(x, df=6, lower_bound=0.1, upper_bound=0.9)' (columns 1:7)
  (to view full data, use np.asarray(this_obj))

In [ ]:
# Create spline lines for 50 evenly spaced values of age
# line_cubic = fit_cubic.predict(dmatrix('bs(xp, knots=(0.33, 0.66))', {'xp': x}))
# line_natural = fit_natural.predict(dmatrix('cr(xp, df=6)', {'xp': x}))

# natural cubic spline
H = np.asarray(x_natural)
sigma = 1
m_Sigma = sigma * sigma * (inv(np.matmul(H.transpose(), H)))
m_cubic_natural = multi_dot([H, m_Sigma, H.transpose()])
res_cubic_natural = m_cubic_natural.diagonal()

# cubic spline
H = np.asarray(x_cubic)
sigma = 1
m_Sigma = sigma * sigma * (inv(np.matmul(H.transpose(), H)))
m_cubic = multi_dot([H, m_Sigma, H.transpose()])
res_cubic = m_cubic.diagonal()

In [2]:
# import plotly.graph_objects as go
# # Create traces
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=x, y=pt_var,
#                         mode='lines+markers',
#                         name='Global Linear'))
# fig.add_trace(go.Scatter(x=x, y=pt_var_cubic,
#                         mode='lines+markers',
#                         name='Global Cubic Spline'))
# fig.add_trace(go.Scatter(x=x, y=res_cubic,
#                         mode='lines+markers', name='Cubic Spline - 2 knots'))
# fig.add_trace(go.Scatter(x=x, y=res_cubic_natural,
#                         mode='lines+markers', name='Natural Cubic Spline - 6 knots'))

# fig.update_layout(
#     xaxis_title="x",
#     yaxis_title="Pointwise Variance",
# )

# fig.update_layout(legend=dict(
#     yanchor="top",
#     y=0.99,
#     xanchor="center",
#     x=0.5
# ))
# fig.show()